# What is Transfer Learning?

Transfer learning, is a research problem in machine learning that focuses on storing knowledge gained while solving one problem and applying it to a different but related problem

# Why Transfer Learning?

- In practice a very few people train a Convolution network from scratch (random initialisation) because it is rare to get enough dataset. So, using pre-trained network weights as initialisations or a fixed feature extractor helps in solving most of the problems in hand.
- Very Deep Networks are expensive to train. The most complex models take weeks to train using hundreds of machines equipped with expensive GPUs.
- Determining the topology/flavour/training method/hyper parameters for deep learning is a black art with not much theory to guide you.

# How Transfer Learning helps ?


When you look at what these Deep Learning networks learn, they try to detect edges in the earlier layers, Shapes in the middle layer and some high level data specific features in the later layers. These trained networks are generally helpful in solving other computer vision problems. Lets have a look at how to do transfer learning using Keras and various cases in Transfer learning.

Q. Implement classification of CIFAR10 dataset using transfer learning (using VGG model   with ImageNet weights). Keeping the 4 CNN blocks fixed, FINETUNE the last block and train the final neural network classifier. This was also illustrated in the class. Try to get an accuracy above 90%

In [0]:
from keras.applications import vgg19
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,Flatten
from keras.datasets import cifar10
from keras.utils import np_utils

In [0]:
(xtrain,ytrain),(xtest,ytest)=cifar10.load_data()
xtrain = xtrain.astype('float32')          #float conversion
xtest = xtest.astype('float32')
xtrain = xtrain/255         #specific preprocessing applied 
xtest = xtest/255
ytrain = np_utils.to_categorical(ytrain, 10)  #conversion of output into matrix
ytest = np_utils.to_categorical(ytest, 10)

In [30]:
# transfer learning model
model=vgg19.VGG19(weights="imagenet",include_top=False,input_shape=(32,32,3))

# adding custom layers using Functional model api of keras
x= model.output
x= Flatten()(x)
x= Dense(256,activation='relu')(x)
x = Dropout(0.5)(x)
x= Dense(125,activation='relu')(x)
pred=Dense(10,activation="softmax")(x)

#creating the final model
model=Model(inputs=model.input,outputs=pred)

# setting no. of untrained layers in our final model
for layer in model.layers[:5]:
    layer.trainable = False
    
model.compile(loss="categorical_crossentropy",
              optimizer="Adadelta",
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

In [32]:
model.fit(xtrain, ytrain,
          batch_size=128,
          epochs=25,
          verbose=1,
          validation_data=(xtest, ytest))

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 31s 625us/step - loss: 2.0881 - acc: 0.1792 - val_loss: 1.7826 - val_acc: 0.2561
Epoch 2/25
50000/50000 [==============================] - 31s 625us/step - loss: 1.7443 - acc: 0.2923 - val_loss: 1.5286 - val_acc: 0.3990
Epoch 3/25
50000/50000 [==============================] - 32s 630us/step - loss: 1.4943 - acc: 0.4278 - val_loss: 1.4928 - val_acc: 0.4280
Epoch 4/25
50000/50000 [==============================] - 32s 635us/step - loss: 1.2394 - acc: 0.5555 - val_loss: 1.2734 - val_acc: 0.5557
Epoch 5/25
50000/50000 [==============================] - 32s 639us/step - loss: 1.0645 - acc: 0.6425 - val_loss: 1.0166 - val_acc: 0.6672
Epoch 6/25
50000/50000 [==============================] - 32s 637us/step - loss: 0.8976 - acc: 0.7078 - val_loss: 1.0983 - val_acc: 0.6378
Epoch 7/25
50000/50000 [==============================] - 32s 636us/step - loss: 0.7815 - acc: 0.7497 - val_loss: 0.

In [33]:
score = model.evaluate(xtest, ytest, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.8845458028793335
Test accuracy: 0.8135
